In [1]:
from cloudsafe import *
import folium
import matplotlib.pyplot as plt
import matplotlib as mpl
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from utils.folium_utils import get_copenhagen_grid
database = 'here-api/main.db'
conn = create_connection(database)
stations = select_stations_join_departures(conn)

In [9]:
def get_grid(start_hour):
    end_hour = start_hour + 1
    if (start_hour == 23):
        end_hour = 0
    f_start_hour = '{}:00'.format(str(start_hour))
    f_end_hour = '{}:00'.format(str(end_hour))
    stations_2 = filter_by_hour(stations, f_start_hour, f_end_hour)
    return get_copenhagen_grid(stations_2)
    # return stations_2['time'] # use for checking if filter works

# maybe possible to make it faster
m = interact(get_grid, start_hour=widgets.IntSlider(min=0, max=23, step=1, value=0))

interactive(children=(IntSlider(value=0, description='start_hour', max=23), Output()), _dom_classes=('widget-i…

In [2]:
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)
stations[['latitude', 'longitude']] = stations.apply(get_coords, axis=1, result_type="expand")

In [3]:
frequencies = stations.name_overpass.value_counts()

In [4]:
stations

,id,id_overpass,name_overpass,coordinates_overpass,id_here,name_here,coordinates_here,no_data,duplicate,id,station_id,bus,headsign,day,time,latitude,longitude
3,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0,0,4,1,12,Islev Viemosevej,saturday,1900-01-01 08:23:00,55.699627,12.469424
4,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0,0,5,1,12,Islev Viemosevej,saturday,1900-01-01 08:53:00,55.699627,12.469424
55,2,node/8084956,Grøndal Station,"55.690687,12.516191",205078064,Grøndal St. (Godthåbsvej),"55.690604,12.516298",0,0,56,2,2A,Refshaleøen,saturday,1900-01-01 08:04:00,55.690604,12.516298
56,2,node/8084956,Grøndal Station,"55.690687,12.516191",205078064,Grøndal St. (Godthåbsvej),"55.690604,12.516298",0,0,57,2,2A,Refshaleøen,saturday,1900-01-01 08:14:00,55.690604,12.516298
57,2,node/8084956,Grøndal Station,"55.690687,12.516191",205078064,Grøndal St. (Godthåbsvej),"55.690604,12.516298",0,0,58,2,2A,Refshaleøen,saturday,1900-01-01 08:24:00,55.690604,12.516298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284882,1046,node/7178922088,Kildebrøndevej,"55.705127,12.478628",205079023,Kildebrøndevej (Husumvej),"55.705084,12.478634",0,0,284883,1046,22,Glostrup St. Brøndbyvester,weekday,1900-01-01 08:30:00,55.705084,12.478634
284883,1046,node/7178922088,Kildebrøndevej,"55.705127,12.478628",205079023,Kildebrøndevej (Husumvej),"55.705084,12.478634",0,0,284884,1046,22,Glostrup St. Brøndbyvester,weekday,1900-01-01 08:50:00,55.705084,12.478634
284930,1047,node/7178922089,Korsager Allé,"55.708405,12.477656",205079022,Korsager Allé (Husumvej),"55.708367,12.477652",0,0,284931,1047,22,Glostrup St. Brøndbyvester,weekday,1900-01-01 08:09:00,55.708367,12.477652
284931,1047,node/7178922089,Korsager Allé,"55.708405,12.477656",205079022,Korsager Allé (Husumvej),"55.708367,12.477652",0,0,284932,1047,22,Glostrup St. Brøndbyvester,weekday,1900-01-01 08:29:00,55.708367,12.477652


In [5]:
frequencies

Peter Bangs Vej                   150
Sølvtorvet                        137
Elmegade                          120
Polititorvet                      120
Bellahøj                          119
                                 ... 
Langelinekaj (Midt)                 3
Hovedbanegården (Tietgensgade)      3
Ålekistevej, Sløjfen                2
Prags Boulevard                     2
Uplandsgade                         2
Name: name_overpass, Length: 409, dtype: int64

In [6]:
rows = []
for station in frequencies.keys():
    first = stations.loc[stations['name_overpass'] == station].iloc[0]
    rows.append({'station_name': station,
                 'longitude': first.longitude,
                 'latitude': first.latitude,
                 'bus_count': frequencies.get(station)})

final = pd.DataFrame(rows)
final['longitude'] = final['longitude'].astype(float)
final['latitude'] = final['latitude'].astype(float)
top_right = [final['latitude'].max(), final['longitude'].max()]
bottom_left = [final['latitude'].min(), final['longitude'].min()]

In [7]:
# Generating a grid heatmap (utils/folium_utils.py)
grid = get_geojson_grid(top_right, bottom_left, n=10)

In [9]:
# Frequencies by latitude and longitude
counts = []
for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]
    items = final.loc[(final['latitude'] <= upper_right[1]) &
                      (final['longitude'] <= upper_right[0]) &
                      (final['longitude'] >= lower_left[0]) &
                      (final['latitude'] >= lower_left[1])]
    counts.append(len(items))
max_count = max(counts)

In [10]:
# Add GeoJson to map
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(counts[i] / max_count)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color': "black",
                            'weight': 2,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.55,
                        })

    m.add_child(gj)

In [11]:
m